## Preambule

In [1]:
import warnings

import numpy as np
from tqdm import tqdm

warnings.simplefilter(action="ignore", category=FutureWarning)

%load_ext autoreload
%autoreload all

## Run classes

Data reader class

In [2]:
from effortsharing.class_datareading import datareading

datareader = datareading("../input.yml")
datareader.read_general()
datareader.read_ssps()
datareader.read_undata()
datareader.read_hdi()
datareader.read_historicalemis_jones()
datareader.read_ar6()
datareader.relation_budget_nonco2()
datareader.determine_global_nonco2_trajectories()
datareader.determine_global_budgets()
datareader.determine_global_co2_trajectories()
datareader.read_baseline()
datareader.read_ndc()
datareader.read_ndc_climateresource()
datareader.merge_xr()
datareader.add_country_groups()
datareader.save()
datareader.country_specific_datareaders()

# ==================================== #
# DATAREADING class                    #
# startyear:  2021
# ==================================== #
- Reading general data
- Reading GDP and population data from SSPs
- Reading UN population data and gapminder, processed by OWID (for past population)
- Read Human Development Index data
- Reading historical emissions (jones)


/home/peter/pbl/effort-sharing/src/effortsharing/class_datareading.py:275: RuntimeWarning: invalid value encountered in cast
  hdi_values_raw = np.array(self.df_hdi_raw.HDI).astype(str)


- Read AR6 data
- Get relationship between CO2 budgets and non-co2 reduction in 2050
- Computing global nonco2 trajectories
- Get global CO2 budgets
- Computing global co2 trajectories
- Reading baseline emissions
- Reading NDC data
- Reading NDC data from Climate resource
- Merging xrarray object
- Add country groups
- Save important files


In [3]:
from effortsharing.class_allocation import allocation

allocator = allocation("USA", lulucf="incl", gas="GHG", settings_file="../input.yml")
allocator.gf()
allocator.pc()
allocator.pcc()
allocator.pcb()
allocator.ecpc()
allocator.ap()
allocator.gdr()
allocator.save()

Allocations specifically for Norway and Netherlands if you want to harmonize to national data

In [5]:
from effortsharing.class_allocation import allocation

for lulucf in ["incl", "excl"]:
    for gas in ["GHG", "CO2"]:
        for dataread_file in [
            "xr_dataread.nc",
            "xr_dataread_NOR.nc",
        ]:  # This is only necessary if for a country a specific historical emissions profile is required beyond what is in Jones (e.g. Norway was requested)
            allocator = allocation(
                "NOR", lulucf=lulucf, gas=gas, dataread_file=dataread_file, settings_file="../input.yml"
            )
            allocator.gf()
            allocator.pc()
            allocator.pcc()
            allocator.pcb()
            allocator.ecpc()
            allocator.ap()
            allocator.gdr()
            allocator.save()
            print("NOR", lulucf, gas, dataread_file, "done")

NOR incl GHG xr_dataread.nc done
NOR incl GHG xr_dataread_NOR.nc done
NOR incl CO2 xr_dataread.nc done
NOR incl CO2 xr_dataread_NOR.nc done
NOR excl GHG xr_dataread.nc done
NOR excl GHG xr_dataread_NOR.nc done
NOR excl CO2 xr_dataread.nc done
NOR excl CO2 xr_dataread_NOR.nc done


Allocation class (run all 207 countries)

In [8]:
from effortsharing.class_allocation import allocation

regions_iso = np.load(
    "../data/output/all_regions.npy", allow_pickle=True
)
for cty in tqdm(regions_iso):
    allocator = allocation(cty, lulucf="incl", gas="GHG", settings_file="../input.yml")
    allocator.gf()
    allocator.pc()
    allocator.pcc()
    allocator.pcb()
    allocator.ecpc()
    allocator.ap()
    allocator.gdr()
    allocator.save()

  8%|▊         | 16/207 [03:50<45:54, 14.42s/it]


KeyboardInterrupt: 

Allocation rules combining approaches (Robiou paper)

In [ ]:
from effortsharing.class_allocation_combinedapproaches import allocation_comb

allocator = allocation_comb()
allocator.ecpc()
allocator.pc()
allocator.discounting_historical_emissions()
allocator.approach1gdp()
allocator.approach1hdi()
allocator.approach2()
allocator.approach2_transition()
allocator.combine()
# allocator.get_relation_2030emis_temp()
allocator.determine_tempoutcomes_discrete()
allocator.save()

Temperature NDC-alignment metric

In [ ]:
from effortsharing.class_tempalign import tempaligning

tempaligner = tempaligning()  # FIRST RUN AGGREGATOR FOR THIS!! (2030 alloc)
tempaligner.get_relation_2030emis_temp()
tempaligner.determine_tempoutcomes()
tempaligner.save()

# ==================================== #
# Initializing tempaligning class        #
# ==================================== #
- Determine relation between 2030-emissions and temperature outcome
- Determine temperature metric


100%|██████████| 6/6 [00:11<00:00,  1.96s/it]


- Save


Reading policy scenarios from ENGAGE

In [ ]:
from effortsharing.class_policyscens import policyscenadding

policyscenner = policyscenadding()
policyscenner.read_engage_data()
policyscenner.filter_and_convert()
policyscenner.add_to_xr()

# ==================================== #
# Initializing policyscenadding class  #
# ==================================== #
- Read ENGAGE scenarios and change region namings
- Filter correct scenarios and convert to xarray object
- Add to overall xrobject


Variance decomposition

In [ ]:
from effortsharing.class_variancedecomp import vardecomposing

vardecomposer = vardecomposing(startyear=2021, gas="GHG", lulucf="incl")
vardecomposer.sobolindices = {}
print("- Starting sobols for fixed years, over many countries")
timeseries = np.arange(2025, 2101, 1)
for year in timeseries:
    print("  Starting with", year)
    xr_cty, ar_time, array_dims, array_inputs, problem, samples = (
        vardecomposer.prepare_global_sobol(year)
    )
    vardecomposer.sobolindices[year] = vardecomposer.apply_decomposition(
        xr_cty, ar_time, array_dims, array_inputs, problem, samples
    )
vardecomposer.save(array_dims, timeseries)

# ==================================== #
# Initializing vardecomposing class    #
# ==================================== #
- Starting sobols for fixed years, over many countries
  Starting with 2025


207it [03:02,  1.13it/s]


  Starting with 2026


207it [02:56,  1.17it/s]


  Starting with 2027


...

207it [02:28,  1.39it/s]


  Starting with 2100


207it [02:27,  1.41it/s]


- Save global results


PermissionError: [Errno 13] Permission denied: 'K:\\Data\\Data_effortsharing\\DataUpdate_ongoing\\startyear_2021\\xr_sobol_GHG_incl.nc'

In [ ]:
vardecomposer.save(array_dims, timeseries)

- Save global results
